In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from scipy.stats import spearmanr
from scipy.stats import skew, kurtosis, norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted
from sklearn.impute import SimpleImputer
import plotly.express as px
import plotly.graph_objects as go
from category_encoders import OneHotEncoder
from IPython.display import VimeoVideo
from sklearn.linear_model import LinearRegression, Ridge  



import plotly.express as px

In [2]:
df = pd.read_csv(r"C:\Users\Nassoh\Desktop\DataSienceMyProject\DS salary\Data\ds_salaries.csv")
print(df.shape)
df.head()

(3755, 11)


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [3]:
df = df.drop(columns=[col for col in df.columns if col not in ['salary_in_usd','work_year', 'experience_level', 'company_size', 'job_title', 'employee_residence']])
df

,work_year,experience_level,job_title,salary_in_usd,employee_residence,company_size
0,2023,SE,Principal Data Scientist,85847,ES,L
1,2023,MI,ML Engineer,30000,US,S
2,2023,MI,ML Engineer,25500,US,S
3,2023,SE,Data Scientist,175000,CA,M
4,2023,SE,Data Scientist,120000,CA,M
...,...,...,...,...,...,...
3750,2020,SE,Data Scientist,412000,US,L
3751,2021,MI,Principal Data Scientist,151000,US,L
3752,2020,EN,Data Scientist,105000,US,S
3753,2020,EN,Business Data Analyst,100000,US,L


# Spilt

In [4]:
target = "salary_in_usd"
y =df[target]
X =df.drop(['salary_in_usd'],axis=1)

In [5]:
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2,random_state=42)

# Baseline

In [9]:
y_mean =y_train.mean()
y_mean
y_pred_baseline =[y_mean]*len(y_train)
len(y_pred_baseline)
mae_baseline =mean_absolute_error(y_train,y_pred_baseline)
print("Mean apt price", round(y_mean, 2))
print("Baseline MAE:", round(mae_baseline, 2))

Mean apt price 138055.99
Baseline MAE: 49169.95


# Build Model

In [10]:
from category_encoders import OrdinalEncoder
from sklearn.neural_network import MLPRegressor
clf =make_pipeline(
    OrdinalEncoder(),
    MLPRegressor(random_state=42)
)
print(clf)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('mlpregressor', MLPRegressor(random_state=42))])


In [11]:
params = {
    "mlpregressor__hidden_layer_sizes": [(50,), (100,), (200,)],
    "mlpregressor__activation": ["relu", "tanh", "logistic"],
    "mlpregressor__solver": ["adam", "sgd"]

}
params

{'mlpregressor__hidden_layer_sizes': [(50,), (100,), (200,)],
 'mlpregressor__activation': ['relu', 'tanh', 'logistic'],
 'mlpregressor__solver': ['adam', 'sgd']}

In [12]:
from sklearn.model_selection import GridSearchCV


model = GridSearchCV(
    clf,
    param_grid=params,
    cv=5,
    n_jobs=-1,

)
model

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                                       ('mlpregressor',
                                        MLPRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'mlpregressor__activation': ['relu', 'tanh',
                                                      'logistic'],
                         'mlpregressor__hidden_layer_sizes': [(50,), (100,),
                                                              (200,)],
                         'mlpregressor__solver': ['adam', 'sgd']})

In [13]:
# Train model
model.fit(X_train,y_train)

c:\Users\Nassoh\.conda\envs\new\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                                       ('mlpregressor',
                                        MLPRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'mlpregressor__activation': ['relu', 'tanh',
                                                      'logistic'],
                         'mlpregressor__hidden_layer_sizes': [(50,), (100,),
                                                              (200,)],
                         'mlpregressor__solver': ['adam', 'sgd']})

In [14]:
cv_results =pd.DataFrame(model.cv_results_)
cv_results_sorted = cv_results.sort_values(by='rank_test_score')
cv_results_sorted

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlpregressor__activation,param_mlpregressor__hidden_layer_sizes,param_mlpregressor__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,5.199603,0.765831,0.025202,0.000982,relu,"(200,)",adam,"{'mlpregressor__activation': 'relu', 'mlpregre...",9.526699e-03,5.508654e-03,6.858079e-03,4.380111e-03,-1.085113e-03,5.037686e-03,3.510212e-03,1
2,3.012202,0.513421,0.027999,0.004098,relu,"(100,)",adam,"{'mlpregressor__activation': 'relu', 'mlpregre...",4.601585e-03,2.214232e-03,3.365830e-03,1.566964e-03,-6.454290e-03,1.058865e-03,3.896498e-03,2
7,0.258000,0.013491,0.031802,0.009661,tanh,"(50,)",sgd,"{'mlpregressor__activation': 'tanh', 'mlpregre...",-2.184872e-03,-2.064860e-03,-2.716233e-03,-1.365131e-04,-7.671846e-03,-2.954865e-03,2.515351e-03,3
0,2.197802,0.287263,0.030800,0.013271,relu,"(50,)",adam,"{'mlpregressor__activation': 'relu', 'mlpregre...",-7.864374e-04,-2.264849e-03,-7.068438e-05,-2.328939e-03,-9.623074e-03,-3.014797e-03,3.415715e-03,4
15,0.489200,0.166051,0.029200,0.003124,logistic,"(100,)",sgd,"{'mlpregressor__activation': 'logistic', 'mlpr...",-3.727725e-03,-9.816921e-03,-2.303446e-03,-2.668131e-03,-1.575341e-02,-6.853926e-03,5.214998e-03,5
13,0.429200,0.169431,0.019599,0.000490,logistic,"(50,)",sgd,"{'mlpregressor__activation': 'logistic', 'mlpr...",-3.860026e-03,-3.642948e-03,-2.430608e-04,-3.000013e-03,-2.851494e-02,-7.852198e-03,1.041198e-02,6
17,0.905202,0.157254,0.026397,0.006560,logistic,"(200,)",sgd,"{'mlpregressor__activation': 'logistic', 'mlpr...",-4.892795e-03,-7.037581e-02,-4.308859e-03,-6.101531e-03,-1.020624e-02,-1.917705e-02,2.568214e-02,7
9,0.433801,0.174973,0.030801,0.005491,tanh,"(100,)",sgd,"{'mlpregressor__activation': 'tanh', 'mlpregre...",-2.171940e-02,-1.257057e-03,-5.648542e-03,-2.997255e-02,-4.654487e-02,-2.102848e-02,1.648643e-02,8
11,0.926200,0.242796,0.034800,0.008565,tanh,"(200,)",sgd,"{'mlpregressor__activation': 'tanh', 'mlpregre...",-2.848179e-01,-2.587146e-01,-5.783570e-02,-6.859019e-02,-5.144778e-02,-1.442812e-01,1.045615e-01,9
10,8.983999,0.877506,0.029200,0.007959,tanh,"(200,)",adam,"{'mlpregressor__activation': 'tanh', 'mlpregre...",-4.898819e+00,-4.938581e+00,-4.696696e+00,-4.788450e+00,-4.481066e+00,-4.760722e+00,1.635729e-01,10


In [15]:
mask=cv_results['rank_test_score']==1
cv_results[mask]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_mlpregressor__activation,param_mlpregressor__hidden_layer_sizes,param_mlpregressor__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,5.199603,0.765831,0.025202,0.000982,relu,"(200,)",adam,"{'mlpregressor__activation': 'relu', 'mlpregre...",0.009527,0.005509,0.006858,0.00438,-0.001085,0.005038,0.00351,1


In [16]:
# Extract best hyperparameters
model.best_params_

{'mlpregressor__activation': 'relu',
 'mlpregressor__hidden_layer_sizes': (200,),
 'mlpregressor__solver': 'adam'}

In [17]:
model.best_score_

0.0050376860580638285

In [18]:
model.best_estimator_

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['experience_level', 'job_title',
                                      'employee_residence', 'company_size'],
                                mapping=[{'col': 'experience_level',
                                          'data_type': dtype('O'),
                                          'mapping': SE     1
MI     2
EN     3
EX     4
NaN   -2
dtype: int64},
                                         {'col': 'job_title',
                                          'data_type': dtype('O'),
                                          'mapping': Data Engineer                1
Research Scientist           2
Data Analyst                 3
Data Scientist               4
Data Science Consultan...
Data DevOps Engineer        83
Machine Learning Manager    84
BI Data Engineer            85
NaN                         -2
Length: 86, dtype: int64},
                                         {'col': 'employee_residence',
                                          'data_type': dtype('O'),
                                          'mapping': ES      1
US      2
GB      3
CA      4
BO      5
       ..
BA     71
RS     72
BG     73
EE     74
NaN    -2
Length: 75, dtype: int64},
                                         {'col': 'company_size',
                                          'data_type': dtype('O'),
                                          'mapping': M      1
S      2
L      3
NaN   -2
dtype: int64}])),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=(200,), random_state=42))])

# Evaluate

In [19]:
y_pred_training =model.predict(X_train)
y_pred_test =model.predict(X_test)
mae_training =mean_absolute_error(y_train,y_pred_training)
mae_test =mean_absolute_error(y_test,y_pred_test)
print("Baseline MAE:", round(mae_baseline, 2))
print("Training MAE:", round(mae_training, 2))
print("Test MAE:", round(mae_test, 2))


Baseline MAE: 49169.95
Training MAE: 48805.77
Test MAE: 48416.0
